This code provides a list of objects that are viewable in your night sky for a given date + time + location. Specifically, it searches the Messier catalog (https://en.wikipedia.org/wiki/Messier_object), the Planets, and an adjustable list of stars or other Simbad astronomical database objects (https://simbad.u-strasbg.fr/simbad/sim-basicIdent=m33&submit=SIMBAD+search). Some important notes:

- The only things that need to be changed by the user (e.g., your observation time) are in the cell labeled "USER NEEDS TO CHANGE". By default, most are set for the BGO in Halifax, Canada.
  
- You can add known star names or Simbad database objects in the "Add stars or other objects" cell - NOTE they MUST be found in Simbad or errors will occur.

- At the very end, the notebook displays observable objects split by type (e.g., globular cluster, nebula, etc.). These are also saved as a PDF, which can be downloaded (see side left panel). The columns and formatting of the tables are copied from the Wikipedia Messier List table, so some fields are blank for non-Messier objects.

# Imports

In [1]:
import astroquery
import datetime
import pytz
import matplotlib
import matplotlib.pyplot as plt 
import pandas as pd 
import astropy 
from astropy.table import Table
from astropy.coordinates import SkyCoord, name_resolve
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz, SkyCoord, get_body
from astroquery.simbad import Simbad
from matplotlib.backends.backend_pdf import PdfPages

# ***USER NEEDS TO CHANGE:***

In [2]:
### Set your desired observation time (use observation_time=None to default to current time ###
observation_time = datetime.datetime(2026, 1, 28, 19, 45, 0) # Year, Month, Day, Hour, Min, Sec

### Set your timezone and location coordinates (default = Halifax, NS, Canada) ###
observer_timezone = "America/Halifax"
observer_latitude = 44.636 # degrees
observer_longitude = -63.5917 # degrees

### altitude_cutoff = degrees above horizon considered comfortably visible given your instrument & surroundings ###
altitude_cutoff = 15  # degrees (15 = default for BGO-Halifax)

### Set the name for the saved PDF (ends up in the left-hand side panel in Binder) ###
save_name = "observation_targets_Jan28"  

# Functions

In [3]:
def RA_convert(hours, minutes, seconds):
    """
    Convert Right Ascension (RA) from HMS format to decimal degrees.

    Parameters:
        hours (int or float): Hours component of RA.
        minutes (int or float): Minutes component of RA.
        seconds (float): Seconds component of RA.

    Returns:
        float: RA in decimal degrees (0-360°).
    """
    RA_degrees = ((hours + minutes / 60 + seconds / 3600) / 24) * 360

    return RA_degrees


def DEC_convert(deg, arcmin, arcsec):
    """
    Convert Declination (DEC) from degrees–arcminutes–arcseconds (DMS) to decimal degrees, with robust handling of negative values.

    For objects with declinations of minus zero (e.g., -0deg 30' 25''), add the negative to the arcmin input param

    Parameters:
        deg (int or float): Degrees component of DEC.
        arcmin (int or float): Arcminutes component of DEC.
        arcsec (float): Arcseconds component of DEC.

    Returns:
        float: Declination in decimal degrees
    """

    # Determine sign from any negative component (covers edge cases like -0° 30′)
    sign = -1 if (deg < 0 or arcmin < 0 or arcsec < 0) else 1

    # Convert absolute DMS values to decimal degrees and reapply sign
    declination_degree = sign * (abs(deg) + abs(arcmin) / 60 + abs(arcsec) / 3600)
    
    return declination_degree 

    
def get_obj_distance(obj_name):
    """
    Query the Simbad astronomical database for the parallax of a given object and calculate its distance in light-years.

    Parameters:
        obj_name (str): Name of the celestial object to query (e.g., "Sirius").

    Returns:
        float: Distance to the object in light-years. Returns 0 if parallax is unavailable or an error occurs.
    """
    # Add parallax field to Simbad queries (global setting)
    Simbad.add_votable_fields('parallax')

    # Query Simbad for the object
    try: 
        result = Simbad.query_object(obj_name)
        parallax_mas = result['plx_value'][0]  # Parallax in milliarcseconds (mas)
    except Exception as e: 
        print(f"Error with parallax: {e}")   
        parallax_mas = 0

    # Convert parallax to distance: distance (pc) = 1000 / parallax (mas)
    if parallax_mas:
        distance_pc = 1000.0 / parallax_mas  # Distance in parsecs
        return distance_pc * 3.26156  # Convert parsecs to light-years
    else:
        print("Parallax not available for object:", obj_name)
        return 0


def add_planets(observation_targets, latitude, longitude, timezone, local_time):
    """
    Add planet data to the existing observation targets list.

    Parameters:
        observation_targets (Astropy Table): Initial list of observation targets.
        latitude (float): Observer's latitude in degrees.
        longitude (float): Observer's longitude in degrees.
        timezone (str): Observer's local timezone (e.g., "America/Halifax").
        local_time (datetime, optional): The local datetime at which to check visibility. Defaults to current time if None.
        
    Returns:
        Astropy Table: A new Astropy Table including the original Messier objects plus the newly added stars. Each added star includes:
                       - Internal IDs
                       - Name
                       - Classification
                       - Object type
                       - Distance (light-years)
                       - Placeholder magnitude
                       - RA/DEC (decimal degrees)
                       - Placeholder additional info
    """
    new_table = observation_targets.copy()  # Avoid overwriting original table

    planets = ["Mercury", "Venus", "Moon", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]
    planet_type = ["Terrestrial Planet", "Terrestrial Planet", "Moon", "Terrestrial Planet", "Gas Giant", "Gas Giant", "Ice Giant", "Ice Giant"]

    location = EarthLocation(lat=latitude*u.deg, lon=longitude*u.deg)
        
    # Get the local timezone object
    local_tz = pytz.timezone(timezone)
    
    # If no local time is provided, use current time
    if local_time is None:
        local_time = datetime.datetime.now(local_tz)
    else:
        # Ensure provided datetime is timezone-aware
        if local_time.tzinfo is None:
            local_time = local_tz.localize(local_time)
        else:
            local_time = local_time.astimezone(local_tz)
    
    # Convert local time to UTC for astropy
    utc_time = local_time.astimezone(pytz.utc)
    time = Time(utc_time)

    planet_coords = {}
    for i in range(len(planets)):
        body = get_body(planets[i], time, location)
        planet_coords[planets[i]] = body.icrs  # RA/Dec in ICRS frame
        
        # Add a new row to the table with star info and placeholder fields
        new_table.add_row([
            "%s" %planets[i],       # Internal ID
            "-",             # Second placeholder ID
            "-",                    # Object common name
            "%s"%planet_type[i],               # Classification
            "Planet",                 # Object type
            0,               # Distance (ly)
            0,                      # Placeholder for magnitude
            body.icrs.ra.deg,                     # Right Ascension (deg)
            body.icrs.dec.deg,                    # Declination (deg)
            "-"                    # Placeholder for additional info
             ])

    return new_table
    
def add_star_data(observation_targets, star_names):
    """
    Add additional data from stars (or other objects from Simbad) to the existing observation targets list.

    Parameters:
        observation_targets (Astropy Table): Initial list of observation targets.
        star_names (list of str): List of star or object names to query and add to the list. These must exist in Simbad.

    Returns:
        Astropy Table: A new Astropy Table including the original targetd plus the newly added objects. Each added row includes:
                       - Internal IDs
                       - Name
                       - Classification
                       - Object type
                       - Distance (light-years)
                       - Placeholder magnitude
                       - RA/DEC (decimal degrees)
                       - Placeholder additional info
    """
    new_table = observation_targets.copy()  # Avoid modifying original table

    # Loop through the provided star names
    for i, obj_name in enumerate(star_names):
        c = SkyCoord.from_name(obj_name)  # Get RA/DEC coordinates using Simbad
        RA = float(c.ra.degree)           # Right Ascension in decimal degrees
        DEC = float(c.dec.degree)         # Declination in decimal degrees

        # Get object distance in light-years (0 if parallax unavailable)
        obj_dist = get_obj_distance(obj_name)

        # Add a new row to the table with star info and placeholder fields
        new_table.add_row([
            "star_%s" % i,          # Internal ID
            "star_%s" % i,          # Second placeholder ID
            obj_name,               # Object name
            "Binary or Multi-Star", # Classification
            "Star",                 # Object type
            obj_dist,               # Distance (ly)
            0,                      # Placeholder for magnitude
            RA,                     # Right Ascension (deg)
            DEC,                    # Declination (deg)
            "NA"])                    # Placeholder for additional info

    return new_table


def is_object_above_horizon(name, ra, dec, lat, lon, altitude_cutoff, timezone, local_time=None):
    """
    Determine whether a celestial object is above the horizon for a given observer location and time.

    Parameters:
        name (str): Name of the celestial object (for display/logging purposes).
        ra (float): Right Ascension of the object in degrees.
        dec (float): Declination of the object in degrees.
        lat (float): Observer's latitude in degrees.
        lon (float): Observer's longitude in degrees.
        altitude_cutoff: degrees above horizon considered visible
        timezone (str): Observer's local timezone (e.g., "America/Halifax").
        local_time (datetime, optional): The local datetime at which to check visibility. Defaults to current time if None.
        
    Returns:
        tuple: 
            - int: 1 if the object is comfortably above horizon given altitude_cutoff,
                   0 if below comfortable viewing altitude.
            - float: altitude of the object in degrees (if above threshold), otherwise 0.
    """
    # Get the local timezone object
    local_tz = pytz.timezone(timezone)
    
    # If no local time is provided, use current time
    if local_time is None:
        local_time = datetime.datetime.now(local_tz)
    else:
        # Ensure provided datetime is timezone-aware
        if local_time.tzinfo is None:
            local_time = local_tz.localize(local_time)
        else:
            local_time = local_time.astimezone(local_tz)

    # Convert local time to UTC for astropy
    utc_time = local_time.astimezone(pytz.utc)
    time = Time(utc_time)

    # Define the observer's location on Earth
    location = EarthLocation(lat=lat*u.deg, lon=lon*u.deg)

    # Convert the object's RA/Dec coordinates to Alt/Az for this observer/time
    sky_coord = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame='icrs')
    altaz = sky_coord.transform_to(AltAz(obstime=time, location=location))

    # Extract the altitude (angle above the horizon)
    altitude = altaz.alt.deg

    # Thresholds for visibility classification
    if altitude >= altitude_cutoff:
        return 1, altitude  # Object is above the horizon
    else:
        return 0, 0  # Object is below horizon


def objects_viewable_tonight(objects, latitude, longitude, timezone, local_time, altitude_cutoff):
    """
    Filter a table of celestial objects to determine which are currently viewable.
    
    Parameters:
        objects (Astropy Table or similar structure): Table containing object info, must include 'RA', 'DEC', and 'MESSIER_NUM' columns.
        latitude (float): Observer's latitude in degrees.
        longitude (float): Observer's longitude in degrees.
        timezone (str): Observer's local timezone (e.g., "America/Halifax").
        local_time (datetime): Local datetime at which to check visibility.
        altitude_cutoff: degrees above horizon considered visible

    Returns:
        Astropy Table: Filtered table containing only objects that are above the comfortable altitude threshold. Adds an 'altitude' column.
    """
    altitude_list = []     # Store computed altitudes of visible objects
    good_name_list = []    # Store names of objects that are comfortably visible

    # Iterate over all objects in the table
    for i in range(len(objects)):
        ra = objects["RA"][i]
        dec = objects["DEC"][i]
        name = objects["MESSIER_NUM"][i]

        # Check if object is above horizon
        checker, altitude = is_object_above_horizon(name, ra, dec, latitude, longitude, altitude_cutoff, timezone, local_time)

        # If comfortably visible, store name and altitude
        if checker == 1:
            good_name_list.append(name)
            altitude_list.append(altitude)

    # Create a set of visible object names for filtering
    ids_set = set(good_name_list)

    # Filter the original table to include only visible objects
    filtered_table = objects[[row['MESSIER_NUM'] in ids_set for row in objects]]

    # Add the altitude column
    filtered_table["altitude"] = altitude_list

    return filtered_table


def single_object_type(TABLE, obj_type):
    # Function splits final observation targets list into categories (e.g., galaxy, nebulae, etc.)
    new_table = TABLE[TABLE["OBJ_CATEGORY"]==obj_type]
    return new_table


def save_tables_to_pdf(tables, pdf_filename):
    # Open a PDF file to save multiple pages
    with PdfPages(pdf_filename) as pdf:
        # Loop through all tables provided
        for name, table in tables:
            # Create a new figure for each table (standard US Letter size)
            fig, ax = plt.subplots(figsize=(8.5, 11))
            ax.axis('off')  # Hide axes for clean text display

            # Convert the Astropy table to formatted string
            table_str = table.pformat(max_lines=-1, max_width=-1)
            text = "\n".join(table_str)

            # Add table text to the figure, with table title
            ax.text(
                0, 1,                    # Top-left corner of the page
                f"{name}\n\n{text}",     # Table title + content
                fontsize=10,             # Readable font size
                va='top',                # Align text from the top
                family='monospace')      # Monospaced font preserves table formatting
            
            # Save the current figure as a page in the PDF
            pdf.savefig(fig, bbox_inches='tight')

            # Close the figure to free memory
            plt.close(fig)

# Import Messier object list

In [4]:
### Loads messier list file + converts to astropy table ###
messier_file = "Messier Objects.xlsx"
messier_objects = Table.from_pandas(pd.read_excel(messier_file))
messier_objects.sort('MESSIER_NUM')
messier_objects['DISTANCE_LY'] = messier_objects['DISTANCE_LY'].astype(int)

print("Messier Catalog Loaded")

Messier Catalog Loaded


# Add Planets + Moon

In [5]:
### Adds planets + moon to observation targets list ###
observation_targets = add_planets(messier_objects, observer_latitude, observer_longitude, observer_timezone, observation_time)
print("Planets Added")

Planets Added


# Add stars or other objects

In [6]:
'''
Running list of known star names that register with Simbad that may/may not be good for viewing:

star_names = ["Albireo", "Sirius", "Alpha Centauri","Capella","Antares","Castor","Fomalhaut","Arcturus","Betelgeuse",
              "Dubhe","Nunki","Rasalhague","Mizar","Sabik","Sarin","Alcor", "Alphecca","Izar","Rasalgethi", "Sheliak"]
'''

### Adds stars to observation targets list ###
star_names = ["Albireo", "Sheliak", "Mizar", "Rasalgethi"] 
observation_targets = add_star_data(observation_targets, star_names)
print("Stars Added")

Stars Added


# Find which objects are viewable tonight

In [7]:
### Run main functions to determine objects in view tonight ###
objects_in_view_tonight = objects_viewable_tonight(observation_targets, observer_latitude, observer_longitude, 
                                                   observer_timezone, observation_time, altitude_cutoff)
print("Objects Identified")

Objects Identified


# Observable objects split by type

In [8]:
### Forms object-type-specific tables ###
observable_planet = single_object_type(objects_in_view_tonight, "Planet")
observable_star = single_object_type(objects_in_view_tonight, "Star")
observable_OC = single_object_type(objects_in_view_tonight, "Open cluster")
observable_nebula = single_object_type(objects_in_view_tonight, "Nebula")
observable_GC = single_object_type(objects_in_view_tonight, "Globular cluster")
observable_galaxy = single_object_type(objects_in_view_tonight, "Galaxy")

### Sorts tables by apparent magnitude (brightest at top)
observable_star.sort("APP_MAG")
observable_OC.sort("APP_MAG")
observable_nebula.sort("APP_MAG")
observable_GC.sort("APP_MAG")
observable_galaxy.sort("APP_MAG")


In [9]:
observable_planet

MESSIER_NUM,NGC_IC_NUM,COMMON_NAME,OBJ_TYPE,OBJ_CATEGORY,DISTANCE_LY,APP_MAG,RA,DEC,SEASON,altitude
str7,str8,str52,str31,str16,int64,float64,float64,float64,str6,float64
Moon,-,-,Moon,Planet,0,0.0,131.3621916932505,18.043276438495827,-,26.026041398610246
Jupiter,-,-,Gas Giant,Planet,0,0.0,113.08781012646331,21.99137263862919,-,41.57498987806277
Saturn,-,-,Gas Giant,Planet,0,0.0,3.178658886860819,-1.1538343376795561,-,21.775219087205002
Uranus,-,-,Ice Giant,Planet,0,0.0,57.71002813754901,19.934928809004678,-,65.26793736553954
Neptune,-,-,Ice Giant,Planet,0,0.0,1.5933441596763085,-0.7737190414887318,-,21.03478750296822


In [10]:
observable_star

MESSIER_NUM,NGC_IC_NUM,COMMON_NAME,OBJ_TYPE,OBJ_CATEGORY,DISTANCE_LY,APP_MAG,RA,DEC,SEASON,altitude
str7,str8,str52,str31,str16,int64,float64,float64,float64,str6,float64
M40,–,Winnecke 4,Optical Double,Star,510,8.4,185.6,58.0833333333333,spring,22.24995969510172


In [11]:
observable_OC

MESSIER_NUM,NGC_IC_NUM,COMMON_NAME,OBJ_TYPE,OBJ_CATEGORY,DISTANCE_LY,APP_MAG,RA,DEC,SEASON,altitude
str7,str8,str52,str31,str16,int64,float64,float64,float64,str6,float64
M45,–,"Pleiades, Seven Sisters or Subaru",Open cluster,Open cluster,425,1.6,56.75,24.1166666666666,winter,69.33410951874885
M44,NGC 2632,Beehive Cluster or Praesepe,Open cluster,Open cluster,577,3.7,130.025,19.9833333333333,winter,28.253934489145884
M41,NGC 2287,Little Beehive Cluster,Open cluster,Open cluster,2300,4.5,101.75,-19.2666666666666,winter,15.882824217829894
M39,NGC 7092,Pyramid Cluster,Open cluster,Open cluster,824,4.6,323.049999999999,48.4333333333333,autumn,27.648966393814213
M35,NGC 2168,Shoe-Buckle Cluster,Open cluster,Open cluster,2800,5.3,92.225,24.3333333333333,winter,57.24372695052232
M34,NGC 1039,Spiral Cluster,Open cluster,Open cluster,1500,5.5,40.5,42.7833333333333,autumn,75.49981845740905
M48,NGC 2548,–,Open cluster,Open cluster,1500,5.5,123.45,-4.2,winter,15.730276358447009
M50,NGC 2323,Heart-Shaped Cluster,Open cluster,Open cluster,3200,5.9,105.8,-7.66666666666666,winter,23.737602155996928
M67,NGC 2682,King Cobra or Golden Eye Cluster,Open cluster,Open cluster,2770,6.1,132.6,11.8166666666666,winter,20.938314331460738


In [12]:
observable_nebula

MESSIER_NUM,NGC_IC_NUM,COMMON_NAME,OBJ_TYPE,OBJ_CATEGORY,DISTANCE_LY,APP_MAG,RA,DEC,SEASON,altitude
str7,str8,str52,str31,str16,int64,float64,float64,float64,str6,float64
M42,NGC 1976,Great Orion Nebula,H II region nebula,Nebula,1344,4.0,83.85,-4.55,winter,36.59192453997659
M78,NGC 2068,–,Diffuse nebula,Nebula,1600,8.3,86.675,0.05,winter,39.77015092479057
M1,NGC 1952,Crab Nebula,Supernova remnant,Nebula,6500,8.4,83.625,22.0166666666666,winter,60.48741992890952
M43,NGC 1982,De Mairan's Nebula,H II region nebula,Nebula,1600,9.0,83.9,-4.73333333333333,winter,36.402778406542
M97,NGC 3587,Owl Nebula,Planetary nebula,Nebula,2030,9.9,168.7,55.0166666666666,spring,26.543956175828768
M76,NGC 650,Little Dumbbell Nebula,Planetary nebula,Nebula,2500,10.1,25.6,51.5666666666666,autumn,65.93785790849165


In [13]:
observable_GC

MESSIER_NUM,NGC_IC_NUM,COMMON_NAME,OBJ_TYPE,OBJ_CATEGORY,DISTANCE_LY,APP_MAG,RA,DEC,SEASON,altitude
str7,str8,str52,str31,str16,int64,float64,float64,float64,str6,float64
M79,NGC 1904,–,Globular cluster,Globular cluster,41000,7.7,81.125,-23.45,winter,19.404744687697175


In [14]:
observable_galaxy

MESSIER_NUM,NGC_IC_NUM,COMMON_NAME,OBJ_TYPE,OBJ_CATEGORY,DISTANCE_LY,APP_MAG,RA,DEC,SEASON,altitude
str7,str8,str52,str31,str16,int64,float64,float64,float64,str6,float64
M31,NGC 224,Andromeda Galaxy,Spiral galaxy,Galaxy,2540000,3.4,10.45,41.2666666666666,autumn,53.80772012632337
M33,NGC 598,Triangulum/Pinwheel Galaxy,Spiral galaxy,Galaxy,2725000,5.7,23.4749999999999,30.65,autumn,57.93061072408269
M81,NGC 3031,Bode's Galaxy,Spiral galaxy,Galaxy,11800000,6.9,148.9,69.0666666666666,spring,41.46638405125874
M32,NGC 221,Andromeda Satellite #1,Dwarf elliptical galaxy,Galaxy,2490000,8.1,10.7,40.8666666666666,autumn,53.8276230352712
M82,NGC 3034,Cigar Galaxy,Starburst galaxy,Galaxy,11500000,8.4,148.95,69.6833333333333,spring,41.63871050782488
M110,NGC 205,Andromeda Satellite #2,Dwarf elliptical galaxy,Galaxy,2690000,8.5,10.1,41.6833333333333,autumn,53.72248525362985
M77,NGC 1068,Cetus A or Squid Galaxy,Spiral galaxy,Galaxy,47000000,8.9,40.675,0.0333333333333333,autumn,42.13389578814505
M74,NGC 628,Phantom Galaxy[91],Spiral galaxy,Galaxy,30000000,9.4,24.175,15.7833333333333,autumn,48.038930505612775
M109,NGC 3992,Vacuum Cleaner Galaxy,Barred Spiral galaxy,Galaxy,83500000,9.8,179.4,53.3833333333333,spring,20.940870765295003


# Save to PDF

In [15]:
save_tables = [("observable_star", observable_star), ("observable_OC", observable_OC), 
               ("observable_nebula", observable_nebula), ("observable_GC", observable_GC), ("observable_galaxy", observable_galaxy)]
save_tables_to_pdf(save_tables, "%s.pdf"%save_name)